In [1]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

--2024-03-09 08:10:18--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26622 (26K) [text/plain]
Saving to: 'edgar_allan_poe.txt'

     0K .......... .......... .....                           100%  980K=0,03s

2024-03-09 08:10:18 (980 KB/s) - 'edgar_allan_poe.txt' saved [26622/26622]

--2024-03-09 08:10:18--  https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP requ

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [5]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [6]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [7]:
len(Ytrain), len(Ytest)

(1618, 540)

In [8]:
train_text[:5]

['i wonder where your oracle is tending',
 'of getting home again because',
 'weve been here fifteen years',
 'i want to introduce you to the people',
 'its the old captains dark fate']

In [9]:
Ytrain[:5]

[1, 1, 1, 1, 1]

In [10]:
idx = 1
word2idx = {'<unk>': 0}

In [11]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [12]:
word2idx

{'<unk>': 0,
 'i': 1,
 'wonder': 2,
 'where': 3,
 'your': 4,
 'oracle': 5,
 'is': 6,
 'tending': 7,
 'of': 8,
 'getting': 9,
 'home': 10,
 'again': 11,
 'because': 12,
 'weve': 13,
 'been': 14,
 'here': 15,
 'fifteen': 16,
 'years': 17,
 'want': 18,
 'to': 19,
 'introduce': 20,
 'you': 21,
 'the': 22,
 'people': 23,
 'its': 24,
 'old': 25,
 'captains': 26,
 'dark': 27,
 'fate': 28,
 'she': 29,
 'thinks': 30,
 'if': 31,
 'it': 32,
 'was': 33,
 'bad': 34,
 'live': 35,
 'with': 36,
 'him': 37,
 'dont': 38,
 'spend': 39,
 'night': 40,
 'outdoors': 41,
 'undo': 42,
 'what': 43,
 'hed': 44,
 'done': 45,
 'and': 46,
 'make': 47,
 'right': 48,
 'life': 49,
 'not': 50,
 'yet': 51,
 'all': 52,
 'gone': 53,
 'out': 54,
 'then': 55,
 'for': 56,
 'house': 57,
 'that': 58,
 'no': 59,
 'more': 60,
 'a': 61,
 'come': 62,
 'up': 63,
 'through': 64,
 'lair': 65,
 'lion': 66,
 'he': 67,
 'said': 68,
 'dead': 69,
 'had': 70,
 'souls': 71,
 'but': 72,
 'when': 73,
 'asked': 74,
 'destination': 75,
 'destin

In [13]:
len(word2idx)

2520

In [14]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens] # get as a word might not be in the populated word2idx (populated from train set)
  test_text_int.append(line_as_int)

In [15]:
train_text_int[100:105]

[[21, 68, 391, 392, 22, 393, 394, 395],
 [396, 70, 53, 397, 353, 33, 306],
 [226, 31, 1, 398, 399, 21, 19, 22, 400],
 [22, 159, 160, 1, 33, 401, 19],
 [402, 69, 83, 290, 403]]

In [17]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [18]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        pi[idx] += 1 # first row in sentence
      else:
        A[last_idx, idx] += 1 # Count as transition

      last_idx = idx # update last idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [19]:
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [20]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [21]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3442521631644005, 0.6557478368355995)

In [25]:
# build a classifier
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors) # number of classes

    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                logprob += logpi[idx]
            else:
                logprob += logA[last_idx, idx]
            last_idx = idx
        return logprob

    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                          for c in range(self.K)]
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions

    

In [26]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [27]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9975278121137207


In [28]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8277777777777777


In [30]:
from sklearn.metrics import confusion_matrix, f1_score

In [31]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 553,    4],
       [   0, 1061]], dtype=int64)

In [32]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[ 80,  85],
       [  8, 367]], dtype=int64)

In [33]:
f1_score(Ytrain, Ptrain)

0.9981185324553152

In [34]:
f1_score(Ytest, Ptest)

0.8875453446191052